<a href="https://colab.research.google.com/github/2022ugmm081/Dynamic-Parking-Pricing/blob/main/BaselineLinearModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Baseline Linear Model is a simple yet effective pricing strategy designed to dynamically update parking prices based on real-time occupancy levels. The model assumes that price should increase proportionally with parking demand. It uses a streaming dataset of occupancy and capacity, and applies the equation:

Price
𝑡
+
1
=
Price
𝑡
+
𝛼
⋅
(
Occupancy/
Capacity
)

This formula captures fluctuations in demand using a tunable sensitivity parameter (α), which controls how fast prices respond to changes in occupancy. The model was implemented in Python using NumPy, and deployed using Pathway's stream simulator (replay_csv), allowing for real-time, daily price updates. Results were visualized using Bokeh, showcasing how prices respond predictably to demand surges.

In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import pathway as pw
import bokeh.plotting
import panel as pn
from pathway import udf

In [4]:
df = pd.read_csv("dataset.csv")

# Merge date + time to Timestamp
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                  format="%d-%m-%Y %H:%M:%S")

# Sort and save simplified streaming CSV
df = df.sort_values("Timestamp").reset_index(drop=True)
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("stream_input.csv", index=False)

In [5]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int

In [6]:
# Simulate a real-time stream from static CSV
data = pw.demo.replay_csv("stream_input.csv", schema=ParkingSchema, input_rate=100)

In [7]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [10]:
class DynamicPricingModel:
    def __init__(self):
        self.alpha = 1.0
        self.prev_price = 10.0
        self.current_day = None
        self.x_vals = []
        self.deltas = []

    def update_price(self, occupancy, capacity, day):
        if capacity == 0:
            return self.prev_price

        # Reset daily
        if day != self.current_day:
            self.prev_price = 10.0
            self.current_day = day
            self.x_vals.clear()
            self.deltas.clear()

        x = occupancy / capacity
        new_price = self.prev_price + self.alpha * x
        delta = new_price - self.prev_price

        self.x_vals.append(x)
        self.deltas.append(delta)

        # Fit α using linear regression if we have 5+ points
        if len(self.x_vals) >= 5:
            try:
                X = np.array(self.x_vals).reshape(-1, 1)
                Y = np.array(self.deltas)
                X_aug = np.hstack([X, np.ones_like(X)])
                coeffs = np.linalg.lstsq(X_aug, Y, rcond=None)[0]
                fitted_alpha = float(np.clip(coeffs[0], 0, 5))
                self.alpha = fitted_alpha
                print(f"🔁 α updated to {self.alpha:.4f}")
            except Exception as e:
                print("⚠️ α update error:", e)

            self.x_vals.clear()
            self.deltas.clear()

        self.prev_price = new_price
        return new_price


In [11]:
# Instantiate model
model = DynamicPricingModel()

# Windowing: daily tumbling + reduction
import datetime

daily_price_stream = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        day = pw.reducers.any(pw.this.day),
        Occupancy = pw.reducers.avg(pw.this.Occupancy),
        Capacity = pw.reducers.max(pw.this.Capacity),
    )
    .with_columns(
        PredictedPrice = pw.apply(
            model.update_price,
            pw.this.Occupancy,
            pw.this.Capacity,
            pw.this.day
        )
    )
)


In [12]:
pn.extension()

def plot_price(source):
    fig = bokeh.plotting.figure(
        height=600,
        width=1200,
        title="📈 Dynamic Parking Price (Priceₜ₊₁ = Priceₜ + α·Occ/Cap)",
        x_axis_type="datetime"
    )
    fig.line("t", "PredictedPrice", source=source, line_width=2, color="green")
    fig.circle("t", "PredictedPrice", source=source, size=6, color="red")
    return fig

viz = daily_price_stream.plot(plot_price, sorting_col="t")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
pw.run()

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input            11                                     
    PythonRead…            1          5644        15833    output           11          0                          
                                                                                                                   
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input             5                                     
    PythonRead…            1          5643        16003    output            5          0                          
                                                                                                                   
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input             6                                     
    PythonRead…            1          5644        16041    output            6          0                          
                                                                                                                   
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input             6                                     
    PythonRead…            1          5645        16098    output            6          0                          
                                                                                                                   
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input             7                                     
    PythonRead…            1          5644        16173    output            7          0                          
                                                                                                                   
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

...

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input             3                                     
    PythonRead…            1          5645        16307    output            3          0                          
                                                                                                                   
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input             3                                     
    PythonRead…            1          5645        16344    output            3          0                          
                                                                                                                   
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input             1                                     
    PythonRead…            1          5644        16382    output           15         14                          
                                                                                                                   
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input             5                                     
    PythonRead…            1          5646        16401    output            5          0                          
                                                                                                                   
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input             6                                     
    PythonRead…            1          5645        16420    output            6          0                          
                                                                                                                   
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    